In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import shuffle
from sklearn import metrics, preprocessing
import gc
#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)
#设置value的显示长度为100，默认为50
pd.set_option('max_colwidth',100)

In [2]:
user_info = pd.read_csv('/media/zsy/Data/ZSY/hw_dataset/data/clean_user_info.csv')
ad_info = pd.read_csv('/media/zsy/Data/ZSY/hw_dataset/data/clean_ad_info.csv')
content_info = pd.read_csv('/media/zsy/Data/ZSY/hw_dataset/data/clean_contentId_info.csv')

In [3]:
user_info.head()

,uId,age,gender,city,province,phoneType,carrier
0,u129897413,4.0,1.0,184.0,25.0,487.0,3.0
1,u108906458,4.0,1.0,125.0,28.0,217.0,2.0
2,u125870461,3.0,1.0,184.0,25.0,474.0,3.0
3,u133924361,4.0,1.0,184.0,25.0,502.0,2.0
4,u133924360,4.0,1.0,184.0,25.0,502.0,3.0


In [4]:
ad_info.head()

,adId,billId,primId,creativeType,intertype,spreadAppId
0,5223,1,6,3,2,73.0
1,5222,1,6,3,2,73.0
2,5221,1,234,3,2,73.0
3,5220,1,234,8,2,73.0
4,5219,1,234,3,2,73.0


In [5]:
content_info.head()

,contentId,firstClass,secondClass
0,5198,23,89
1,5197,1,1
2,5196,1,1
3,5195,1,1
4,5194,1,1


In [1]:
del train
del neg
del pos
del train_uid1_label1_counts_data
del train_uid_label0_counts_data
gc.collect()

# 处理每一天的数据，按照uid出现次数采样，label＝１
# train_28.csv代表不同的天数

In [15]:
train = pd.read_csv(r'/media/zsy/Data/ZSY/hw_dataset/data/init/26-31/train_28.csv')

In [16]:
train.columns = ['label', 'uId', 'adId', 'operTime', 'siteId', 'slotId', 'contentId', 'netType']

# 分离出train中label = 1 和　lable = 0 的数据
# 并分别计算他们的uid 出现的次数，作为新的一列

In [17]:
neg = train.loc[train['label'] == 0]
pos = train.loc[train['label'] == 1]
neg = neg.reset_index(drop = True)
pos = pos.reset_index(drop = True)

In [18]:
neg.shape

(23331611, 8)

In [19]:
pos.shape

(1475854, 8)

In [20]:
uid_stats = pos['uId'].value_counts()
pos_uid_counts = np.hstack([pos['uId'].values.reshape(-1,1),pos['uId'].map(uid_stats).values.reshape(-1,1)])
pos_uid_counts = pd.DataFrame(pos_uid_counts)
pos_uid_counts.columns = ['uId','counts']
pos['uId_count'] = pos_uid_counts['counts']

In [21]:
uid_stats = neg['uId'].value_counts()
neg_uid_counts = np.hstack([neg['uId'].values.reshape(-1,1),neg['uId'].map(uid_stats).values.reshape(-1,1)])
neg_uid_counts = pd.DataFrame(neg_uid_counts)
neg_uid_counts.columns = ['uId','counts']
neg['uId_count'] = neg_uid_counts['counts']

# 将label = 1 的数据中，uid仅出现过一次的那些数据提取出来

In [22]:
train_uid1_label1_counts_data = pos.loc[pos['uId_count'] == 1,:]
train_uid1_label1_counts_data = train_uid1_label1_counts_data.reset_index(drop = True)
train_uid1_label1_counts_data.shape

(761090, 9)

# 在label = 0 的数据中，使用上一步得到的数据的uid进行匹配，得到label = 0 的数据

In [23]:
train_uid_label0_counts_data = neg.loc[neg['uId'].isin(train_uid1_label1_counts_data['uId'].unique().tolist()),:]
train_uid_label0_counts_data = train_uid_label0_counts_data.reset_index(drop = True)
train_uid_label0_counts_data.shape

(2628280, 9)

In [24]:
rest_num = 1000000-train_uid1_label1_counts_data.shape[0]

# 采样剩下的负样本，凑够１００万数据

In [69]:
index = np.random.randint(0,train_uid_label0_counts_data.shape[0],rest_num)
train_uid_label0_counts_sample = train_uid_label0_counts_data.iloc[index,:]

# 进行数据融合，并且打乱顺序

In [70]:
data_label1_uid1 = pd.concat([train_uid1_label1_counts_data,train_uid_label0_counts_sample])
data_label1_uid1 = data_label1_uid1.reset_index(drop = True)
data_label1_uid1.drop(['uId_count'],axis = 1,inplace = True)
data_label1_uid1 = shuffle(data_label1_uid1)
data_label1_uid1 = data_label1_uid1.reset_index(drop = True)

In [71]:
uid_stats = data_label1_uid1['uId'].value_counts()
train_uid1_counts = np.hstack([data_label1_uid1['uId'].values.reshape(-1,1),data_label1_uid1['uId'].map(uid_stats).values.reshape(-1,1)])
train_uid1_counts = pd.DataFrame(train_uid1_counts)
train_uid1_counts.columns = ['uId','counts']
data_label1_uid1['uId_count'] = train_uid1_counts['counts']

In [72]:
data_label1_uid1.head()

,label,uId,adId,operTime,siteId,slotId,contentId,netType,uId_count
0,1,u126919839,1582,2019-03-31 12:39:05.245,10,8,1222,1,1
1,1,u143373065,3408,2019-03-31 18:40:48.623,8,87,4005,1,1
2,1,u148060321,3887,2019-03-31 08:30:45.805,10,30,2137,1,1
3,1,u134582615,1614,2019-03-31 19:41:25.356,9,22,1676,1,1
4,1,u120865155,3486,2019-03-31 22:15:03.475,8,88,4104,4,2


# 最终数据融合

In [73]:
data_label1_uid1 = pd.merge(data_label1_uid1,user_info, how='left', left_on='uId',right_on='uId')
data_label1_uid1 =  pd.merge(data_label1_uid1,ad_info, how='left', left_on='adId',right_on='adId')
data_label1_uid1 =  pd.merge(data_label1_uid1,content_info, how='left', left_on='contentId',right_on='contentId')
data_label1_uid1 = data_label1_uid1.fillna(0)

In [5]:
# test只需运行一次，生成一个test样本就可以了
test_data = pd.merge(test_data,user_info, how='left', left_on='uId',right_on='uId')
test_data =  pd.merge(test_data,ad_info, how='left', left_on='adId',right_on='adId')
test_data =  pd.merge(test_data,content_info, how='left', left_on='contentId',right_on='contentId')
test_data = data_label1_uid1.fillna(0)
test_data.to_csv('test.csv',index = False)

In [6]:
# 存储数据

In [75]:
data_label1_uid1.to_csv('data/by_data/uid1_label1_train_28.csv',index = False)